### scrapy
- 웹사이트에서 데이터 수집을 위한 오픈소스 파이썬 프레임워크
- 멀티스레딩으로 데이터 수집
- gmarket 상품데이터 수집

In [2]:
# 1. 스크래피 프로젝트 생성

In [1]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\code\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [2]:
!tree /f gmarket

폴더 PATH의 목록입니다.
볼륨 일련 번호가 0000004F A0EF:7F3F입니다.
C:\USERS\USER\CODE\GMARKET
│  scrapy.cfg
│  
└─gmarket
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


In [6]:
# items : 데이터의 모양 정의
# middewares : 수집할때 header 정보와 같은 내용을 설정
# pipelines : 데이터를 수집한 후에 코드를 실행
# settings : robots.txt 규칙, 크롤링 시간 텀등을 설정
# spiders : 크롤링 절차를 정의

In [7]:
# 2. xpath 찾기 : 링크, 상세 페이지

In [3]:
import scrapy, requests
from scrapy.http import TextResponse

In [15]:
# 3. items.py : model

In [16]:
%%writefile gmarket/gmarket/items.py
import scrapy

# 하나의 row 데이터를 담을 클래스
class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [17]:
# 4. spider.py : 크롤링 절차 정의

In [18]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB" # spider 이름
    allow_domain = ["gmarket.co.kr"] # 허용 도메인
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"] # 최초 request URL
    
    def parse(self, response): # 최초 request에 대한 reponse 메서드
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li/div[1]/a/@href').extract() # link 수집 selector
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content) # link로 request 후 response를 받아 callback함수 호출
    
    def parse_content(self, response): # callback 으로 실행 되는 함수
        item = GmarketItem() # 하나의 row 데이터를 담을 객체
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract() # title 수집 selector
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract() # price 수집 selector
        item["link"] = response.url # request에서 사용한 url
        yield item

Writing gmarket/gmarket/spiders/spider.py


In [19]:
# 5. 스크래피 실행
# 주피터 노트북 터미널 실행
# gmarket 디렉토리에서 아래의 커멘드 실행
# scrapy crawl GMB -o items.csv

In [22]:
%pwd

'C:\\Users\\User\\code\\project\\test'

In [21]:
%ls gmarket

 C 드라이브의 볼륨: 로컬 디스크
 볼륨 일련 번호: 8A64-4954

 C:\Users\User\code\project\test\gmarket 디렉터리

2022-02-25  오후 02:13    <DIR>          .
2022-02-25  오후 02:13    <DIR>          ..
2022-02-25  오후 02:13    <DIR>          gmarket
2022-02-25  오후 02:13               257 scrapy.cfg
               1개 파일                 257 바이트
               3개 디렉터리  427,162,914,816 바이트 남음


In [23]:
import pandas as pd

In [25]:
pd.read_csv("gmarket/items.csv").tail(2)

,link,price,title
198,http://item.gmarket.co.kr/Item?goodscode=14781...,"25,700",아기물티슈 최종 21850원 75g 20팩 시그니처 캡형
199,http://item.gmarket.co.kr/Item?goodscode=23227...,"37,710",10%할인 NEW 깨끗한나라 허브가든 벚꽃 프리미엄 화장지 총 90롤
